# McStas data to MJOLNIR data format, conversion script for BIFROST

by: Kristine M. L. Krighaar

created: 18/01/2024

In [12]:
from KGS import * # Utilizing my own package for data analysis. Remember to also send enviroment when sharing. 
import scipp as sc
import scippneutron as scn
from scippneutron.conversion import graph
import os
from IPython.display import Image
import h5py

## Data reduction process

Full overview of how converison is done: 

![conversion_tree](DataReductionToMjolnir.png)

## Loading data from simulation

In [13]:
folder_path = "../McStasScript/data_folder/bifrost_12/"
search_term = "detector"

file_list = [file for file in os.listdir(folder_path) if search_term in file]

I_A3 = []
Ierr_A3 = []

for i in range(2):
    I = []
    I_err = []

    for file in file_list:

        # Extract data from simulation files 
        data = np.loadtxt(folder_path+file).reshape(3, 100, 100)
        I.append(data[0].T)
        I_err.append(data[1].T)

    I = np.asarray(I).T
    I_err = np.asarray(I_err).T
    
    I_A3.append(I)
    Ierr_A3.append(I_err)

I_A3 = np.asarray(I_A3)
Ierr_A3 = np.asarray(Ierr_A3)



In [59]:
def get_subfolders_names(folder):
    subfolders = []

    def collect_subfolders(name, obj):
        if isinstance(obj, h5py.Group):
            subfolders.append(name)

    folder.visititems(collect_subfolders)
    return subfolders


def import_nxs_data(nxs_file_path):

    data_dict = {}
    y__m_dict = {}
    t__s_dict = {}

    with h5py.File(nxs_file_path, 'r') as file:
        #Print the structure of the HDF5 file
        file.visititems(lambda name, obj: print(f"Name: {name}, Type: {type(obj)}"))
        
        folder_path = "entry1/data/"
        folder = file[folder_path]
        subfolders = get_subfolders_names(folder)

        for i in range(len(subfolders)):
            I_data = file[folder_path+subfolders[i]+'/data']
            err_data = file[folder_path+subfolders[i]+'/data']
            I_data = I_data[()]
            data_set = np.array(I_data)
            
            data_dict[subfolders[i]]= I_data
    
    return data_dict
        


data = import_nxs_data('../McStasScript/run_folder/test/mccode.hdf5')

print(data[1])


{'detector_signal_1Dspace_close_1_5p0_1_dat': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'detector_signal_1Dspace_close_1_5p0_4_dat': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'detector_signal_1Dspace_close_1_5p0_7_dat': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'detector_signal_1Dspace_close_2_5p0_2_dat': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0

## Organizing datastructure in scipp 

Datastructure includes lab information

In [3]:
#data_dict = {}
for i in range(2):
    key = f'A3 setting{i}'
    value = sc.array(dims=['y', 't', 'tube'], values=I_A3[i])
    data_dict[key] = value

print(data_dict)

{'A3 setting0': <scipp.Variable> (y: 100, t: 100, tube: 27)    float64  [dimensionless]  [0, 0, ..., 0, 0], 'A3 setting1': <scipp.Variable> (y: 100, t: 100, tube: 27)    float64  [dimensionless]  [0, 0, ..., 0, 0]}


In [4]:
sim_data = sc.Dataset(
    data=data_dict, # sc.array(dims=['y', 't', 'tube'], values=I_A3),
    coords={
        'y': sc.array(dims=['y'], values=np.arange(100.0)),
        'A4': sc.array(dims=['y'], values=np.arange(100.0)),
        'Lsd': sc.array(dims=['y'], values=np.arange(100.0)),
        't': sc.array(dims=['t'], values=np.arange(100.0), unit='s'),
        'tube': sc.array(dims=['tube'], values=np.arange(27.0)),
        'Ana': sc.array(dims=['tube'], values=np.arange(27.0)),
        'wedge': sc.array(dims=['tube'], values=np.arange(27.0)),
    },
)

sc.show(sim_data)
sim_data



<scipp.Dataset>
Dimensions: Sizes[y:100, t:100, tube:27, ]
Coordinates:
* A4                        float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
* Ana                       float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* Lsd                       float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
* t                         float64              [s]  (t)  [0, 1, ..., 98, 99]
* tube                      float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* wedge                     float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* y                         float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
Data:
  A3 setting0               float64  [dimensionless]  (y, t, tube)  [0, 0, ..., 0, 0]
  A3 setting1               float64  [dimensionless]  (y, t, tube)  [0, 0, ..., 0, 0]

# Constructing the data reduction tree

In [7]:
def backscattered_l2(position, sample_position, analyzer_position):
    """
    Compute the length of the secondary flight path for backscattering off an analyzer.
    """
    return sc.norm(position - analyzer_position) + sc.norm(
        analyzer_position - sample_position
    )


def wavelength_from_analyzer(analyzer_dspacing, analyzer_angle):
    """
    Compute the neutron wavelength after scattering from the analyzer's d-spacing.

    Assuming Bragg scattering in the analyzer, the wavelength is
        wavelength = 2 * d * sin(theta)

    Where
        d is the analyzer's d-spacing,
        theta is the scattering angle or equivalently, the tilt of the analyzer
              w.r.t. to the sample-analyzer axis.
    """
    # 2*theta is the angle between transmitted and scattered beam.
    return (
        2
        * analyzer_dspacing
        * sc.sin(sc.scalar(np.pi / 2, unit="rad") - analyzer_angle.to(unit="rad"))
    )

In [9]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.tof import energy_transfer_indirect_from_tof

graph = {
    **beamline(scatter=True),
    "energy_transfer": energy_transfer_indirect_from_tof,
    # Replace L2 with our own implementation.
    "L2": backscattered_l2,
    # Insert a new function for the wavelength.
    "final_wavelength": wavelength_from_analyzer,
}
# Optional: remove unused functions in order to clean up the image below.
del graph["two_theta"]
del graph["scattered_beam"]
del graph["Ltotal"]
#sc.show_graph(graph, simplified=True)